Escrever o código necessário para alterar a caixa dos valores da coluna nome para MAIÚSCULO.
Imprimir a contagem de linhas presentes no dataframe


In [2]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper

spark = SparkSession.builder.appName("MyGlueApp").getOrCreate()
S3_INPUT_PATH = "s3://aws-glue-assets-477879694723-us-east-1/nomes.csv"
df = spark.read.csv(S3_INPUT_PATH, header=True, inferSchema=True)
df = df.withColumn("nome", upper(df["nome"]))

# Imprime o schema do DataFrame
df.printSchema()

# Imprime a contagem de linhas
row_count = df.count()
print("Contagem de linhas no DataFrame:", row_count)


root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)

Contagem de linhas no DataFrame: 1825433


Imprimir a contagem de nomes, agrupando os dados do dataframe pelas colunas ano e sexo.
Ordene os dados de modo que o ano mais recente apareça como primeiro registro do dataframe.


In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, upper, sum

# Inicialize o SparkSession
spark = SparkSession.builder.appName("MyGlueApp").getOrCreate()
S3_INPUT_PATH = "s3://aws-glue-assets-477879694723-us-east-1/nomes.csv"
df = spark.read.csv(S3_INPUT_PATH, header=True, inferSchema=True)

# Altera a caixa dos valores da coluna "nome" para maiúsculo
df = df.withColumn("nome", upper(df["nome"]))

# Agrupa os dados pelas colunas "ano" e "sexo" e soma a coluna "total"
grouped_df = df.groupBy("ano", "sexo").agg(sum("total").alias("total_soma"))

# Ordena os dados pelo ano de forma descendente (do mais recente para o mais antigo)
grouped_df = grouped_df.orderBy(desc("ano"))

grouped_df.show()

+----+----+----------+
| ano|sexo|total_soma|
+----+----+----------+
|2014|   F|   1768775|
|2014|   M|   1901376|
|2013|   F|   1745339|
|2013|   M|   1881463|
|2012|   M|   1889414|
|2012|   F|   1753922|
|2011|   F|   1753500|
|2011|   M|   1893230|
|2010|   M|   1913851|
|2010|   F|   1772738|
|2009|   F|   1832925|
|2009|   M|   1979303|
|2008|   F|   1887234|
|2008|   M|   2036289|
|2007|   F|   1919408|
|2007|   M|   2072139|
|2006|   F|   1898463|
|2006|   M|   2052377|
|2005|   M|   1994841|
|2005|   F|   1845379|
+----+----+----------+
only showing top 20 rows

Apresentar qual foi o nome feminino e masculino com mais registros e em que ano ocorreu.


In [ ]:
from pyspark.sql.functions import desc

# Filtra os registros do sexo feminino
df_feminino = df.filter(df["sexo"] == "F")
# Encontre o nome feminino com mais registros
nome_fem_com_mais_registros = df_feminino.groupBy("nome").agg({"total": "max"}).sort(desc("max(total)")).first()
# Encontra o ano em que o nome feminino mais registrado ocorreu
ano_nome_fem_mais_registrado = df_feminino.filter(df_feminino["nome"] == nome_fem_com_mais_registros["nome"]).groupBy("ano").agg({"total": "max"}).sort(desc("max(total)")).first()["ano"]
# Imprima o resultado
print(f"O nome feminino com mais registros é '{nome_fem_com_mais_registros['nome']}' com um total de {nome_fem_com_mais_registros['max(total)']} registros e ocorreu no ano {ano_nome_fem_mais_registrado}.")

# Filtra os registros do sexo masculino
df_masculino = df.filter(df["sexo"] == "M")
# Encontra o nome masculino com mais registros
nome_masc_com_mais_registros = df_masculino.groupBy("nome").agg({"total": "max"}).sort(desc("max(total)")).first()
# Encontra o ano em que o nome masculino mais registrado ocorreu
ano_nome_masc_mais_registrado = df_masculino.filter(df_masculino["nome"] == nome_masc_com_mais_registros["nome"]).groupBy("ano").agg({"total": "max"}).sort(desc("max(total)")).first()["ano"]
# Imprima o resultado
print(f"O nome masculino com mais registros é '{nome_masc_com_mais_registros['nome']}' com um total de {nome_masc_com_mais_registros['max(total)']} registros e ocorreu no ano {ano_nome_masc_mais_registrado}.")



O nome feminino com mais registros é 'LINDA' com um total de 99680 registros e ocorreu no ano 1947.
O nome masculino com mais registros é 'JAMES' com um total de 94755 registros e ocorreu no ano 1947.

Apresentar o total de registros (masculinos e femininos) para cada ano presente no dataframe.
Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.



In [ ]:
# Importe as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper, sum

# Inicialize o SparkSession
spark = SparkSession.builder.appName("MyGlueApp").getOrCreate()

# Defina o caminho de entrada para o arquivo CSV
S3_INPUT_PATH = "s3://aws-glue-assets-477879694723-us-east-1/nomes.csv"

# Leia o arquivo CSV do S3
df = spark.read.csv(S3_INPUT_PATH, header=True, inferSchema=True)

# Altera a caixa dos valores da coluna "nome" para maiúsculo
df = df.withColumn("nome", upper(df["nome"]))

# Filtra os registros do sexo feminino
df_feminino = df.filter(df["sexo"] == "F")

# Agrupe os dados pelo ano e some a coluna "total"
total_registros_por_ano = df_feminino.groupBy("ano").agg(sum("total").alias("total_registros"))

# Ordene os resultados pelo ano, de forma crescente (não use desc)
total_registros_por_ano = total_registros_por_ano.sort("ano")

# Limite os resultados às primeiras 10 linhas
total_registros_por_ano = total_registros_por_ano.limit(10)

# Imprima os resultados
for registro in total_registros_por_ano.collect():
    print(f"{registro['ano']}: {registro['total_registros']}")


1880: 90993
1881: 91954
1882: 107850
1883: 112321
1884: 129022
1885: 133055
1886: 144535
1887: 145982
1888: 178627
1889: 178366

In [ ]:
Escrever o conteúdo do dataframe com os valores de nome em maiúsculo no S3.
A gravação deve ocorrer no subdiretório frequencia_registro_nomes_eua do path s3://<BUCKET>/lab-glue/
O formato deve ser JSON
O particionamento deverá ser realizado pelas colunas sexo e ano (nesta ordem)

In [8]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper

# Inicialize o SparkSession
spark = SparkSession.builder.appName("MyGlueApp").getOrCreate()

# Defina o caminho de entrada para o arquivo nomes.csv no S3
S3_INPUT_PATH = "s3://aws-glue-assets-477879694723-us-east-1/nomes.csv"

# Leia o arquivo CSV do S3
df = spark.read.csv(S3_INPUT_PATH, header=True, inferSchema=True)

# Altera a caixa dos valores da coluna "nome" para maiúsculo
df = df.withColumn("nome", upper(df["nome"]))

# Defina o caminho de saída no S3 com o particionamento adequado
S3_OUTPUT_PATH = "s3://aws-glue-assets-477879694723-us-east-1/frequencia_registro_nomes_eua/"

# Grave o DataFrame no S3 em formato JSON e com particionamento por sexo e ano
df.write.partitionBy("sexo", "ano").json(S3_OUTPUT_PATH)
